### 安装并设置searxng

- 打开终端，运行 `wsl --install` 安装linux子系统
- 接着安装docker，前往[官网](https://docs.docker.com/desktop/setup/install/windows-install/)下载并安装即可
- 打开docker桌面程序，点击程序上方齿轮形状的设置图标，然后再点击左侧的`Docker Engine`设置栏，最后添加镜像源：`"registry-mirrors": ["https://docker.1ms.run"]`，添加完后即可点击`Apply`变成修改
- 现在请创建如下文件夹：`mkdir -p ./searxng/config/ ./searxng/data/`，接着在searxng文件夹下打开终端，然后下载并运行searxng搜索引擎：`docker run --name searxng --restart=always -d -p 8888:8080 -v "./config/:/etc/searxng/" -v "./data/:/var/cache/searxng/" docker.io/searxng/searxng:latest`。最后打开`http://localhost:8888/`你应该能看到搜索引擎界面了
- 在网页界面点击右上角的设置按钮，在搜索引擎中选择 bing、360search、baidu、sogou，然后保存设置并回到搜索界面，输入搜索内容并回车，你应该能看到返回结果
- 注意上一步得到的结果是html格式的，我们调用接口需要用到json格式，所以打开`./searxng/config/settings.yml`，搜索`formats:`找到格式配置，在下方添加`- json`配置，接着你需要回到docker程序重启容器

In [1]:
from datetime import date
from typing import List

from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.utilities import SearxSearchWrapper
from langchain_core.tools import StructuredTool

In [2]:
# ------------------ 工具 1：联网搜索 ------------------
searx = SearxSearchWrapper(
    searx_host="http://localhost:8888", engines=["bing", "360search", "baidu", "sogou"]
)
search_tool = StructuredTool.from_function(
    func=searx.run, name="search", description="通过多个中文搜索引擎查询实时信息"
)

In [3]:
# ------------------ 工具 2：获取当天日期 ------------------
@StructuredTool.from_function
def today(_: str = "") -> str:
    """返回今天的日期，格式 yyyy-mm-dd"""
    return str(date.today())

In [4]:
# ------------------ 初始化 Agent ------------------
memory = MemorySaver()
model = ChatOllama(model="qwen2.5", temperature=0)
agent_executor = create_react_agent(
    model, tools=[search_tool, today], checkpointer=memory
)

In [5]:
# ------------------ 统一对话封装 ------------------
def run_chat(questions: List[str], thread_id: str = "abc123") -> None:
    """
    批量提问，自动流式输出并打印最终结果。
    :param questions: 用户问题列表
    :param thread_id: 用于记忆隔离
    """
    config = {"configurable": {"thread_id": thread_id}}
    for q in questions:
        for step in agent_executor.stream(
            {"messages": [{"role": "user", "content": q}]},
            config=config,
            stream_mode="values",
        ):
            step["messages"][-1].pretty_print()

In [6]:
# ------------------ 主入口 ------------------
if __name__ == "__main__":
    # 测试用例
    questions = ["今天是几号？", "深圳近期天气"]
    run_chat(questions)

================================ Human Message =================================

今天是几号？
================================== Ai Message ==================================
Tool Calls:
  today (868d6d99-826c-44d3-a296-1b8cc1b4d8c5)
 Call ID: 868d6d99-826c-44d3-a296-1b8cc1b4d8c5
  Args:
================================= Tool Message =================================
Name: today

2025-07-13
================================== Ai Message ==================================

今天是2025年7月13日。
================================ Human Message =================================

深圳近期天气
================================== Ai Message ==================================
Tool Calls:
  search (84f3f71a-8ced-488c-873f-8803f17c6ab7)
 Call ID: 84f3f71a-8ced-488c-873f-8803f17c6ab7
  Args:
    query: 深圳近期天气
================================= Tool Message =================================
Name: search

深圳天气预报,及时准确发布中央气象台天气信息,便捷查询深圳今日天气,深圳周末天气,深圳一周天气预报,深圳蓝天预报,深圳天气预报,深圳40日天气预报,还提供深圳的生活指数、健康指数、交通指数、旅游指数,及时发布深圳气象预警信号、各类气